<a href="https://colab.research.google.com/github/hamzaq453/Langchain-learnings/blob/main/Router.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U langchain_google_genai langchain_core langgraph


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.5/41.5 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.2/137.2 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.3 MB/s eta 0:00:00


In [2]:
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.graph import StateGraph, START, END
from langgraph.graph import MessagesState
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition
from langchain_core.messages import HumanMessage, SystemMessage

# Load GEMINI API Key
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')

# Initialize LLM
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", api_key=GEMINI_API_KEY)


In [9]:
# Tool 1: Multiply
def multiply(a: int, b: int) -> int:
    """Multiplies two numbers."""
    return a * b

# Tool 2: Add
def add(a: int, b: int) -> int:
    """Adds two numbers."""
    return a + b

# Tool 3: Subtract
def subtract(a: int, b: int) -> int:
    """Subtracts two numbers."""
    return a - b

# Bind tools to LLM
llm_with_tools = llm.bind_tools([multiply, add, subtract])


In [10]:
def tool_calling_llm(state: MessagesState) -> MessagesState:
    """Handles LLM invocation and determines if tool call is needed."""
    return {"messages": [llm_with_tools.invoke(state["messages"])]}


In [11]:
# Define the node for tool invocation
def tool_calling_llm(state: MessagesState) -> MessagesState:
    """Handles LLM invocation and determines if tool call is needed."""
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

# Build the graph
builder = StateGraph(MessagesState)

# Add nodes
builder.add_node("tool_calling_llm", tool_calling_llm)
builder.add_node("tools", ToolNode([multiply, add, subtract]))

# Add edges
builder.add_edge(START, "tool_calling_llm")
builder.add_conditional_edges(
    "tool_calling_llm",
    tools_condition,  # Route based on tool condition
)
builder.add_edge("tools", END)

# Compile the graph
graph = builder.compile()


In [13]:
from langchain_core.messages import HumanMessage, SystemMessage
messages = [SystemMessage(content="") , HumanMessage(content="What is a product of 2 and 3 and what is 5 + 3")]
messages = graph.invoke({"messages": messages})
for m in messages['messages']:
    m.pretty_print()

================================ System Message ================================


================================ Human Message =================================

What is a product of 2 and 3 and what is 5 + 3
================================== Ai Message ==================================
Tool Calls:
  multiply (6a440e61-15cc-4a54-959e-6505852e0e2f)
 Call ID: 6a440e61-15cc-4a54-959e-6505852e0e2f
  Args:
    a: 2.0
    b: 3.0
  add (471d87b1-75ab-4e24-9d31-ebdf16856253)
 Call ID: 471d87b1-75ab-4e24-9d31-ebdf16856253
  Args:
    a: 5.0
    b: 3.0
================================= Tool Message =================================
Name: multiply

6
================================= Tool Message =================================
Name: add

8
